In [19]:
# https://pszufe.github.io/OpenStreetMapX_Tutorial/JuliaCon2021/
cd(@__DIR__)
import Pkg
Pkg.activate(".")

  Activating project at `~/Documents/Documents - spectra/Dev/Julia/abm-dsap/code/04-OSM-SIR`


In [31]:
using Random, Parameters, OpenStreetMapX, PyCall, CSV, DataFrames, SparseArrays, LightGraphs, DataStructures, Plots
ENV["GKSwstype"]="nul"  # this parameter significantly speeds up generation of GR-based plots
gr()
using OpenStreetMapXPlot
const flm = pyimport("folium");

In [21]:
# m = get_map_data("maps/george_sq.osm", use_cache=false, trim_to_connected_graph=true );
# https://szufel-public.s3.us-east-2.amazonaws.com/central_torontoF.osm
m = get_map_data("maps/central_torontoF.osm", use_cache=false, trim_to_connected_graph=true );
typeof(m)

MapData

In [22]:
m.g

{179, 354} directed simple Int64 graph

In [23]:
m.n

179-element Vector{Int64}:
             29603223
             26240542
             25628766
             34522600
  3786828693378877191
             29604863
             29604906
             29687986
             29696827
             29605043
                    ⋮
            118410163
            394509546
            281361927
             26240577
             53542723
 -5705736890598083847
           3712797466
           3722123211
             59566700

In [24]:
m.nodes

Dict{Int64, ENU} with 744 entries:
  6532307455 => ENU(-341.765, 565.58, -0.0342666)
  6593471028 => ENU(255.988, 177.729, -0.00760988)
  3700157943 => ENU(-269.491, -584.309, -0.0325004)
  29688214   => ENU(547.111, 352.804, -0.0332043)
  4204756690 => ENU(-126.107, -484.707, -0.0196979)
  394497217  => ENU(401.392, 500.575, -0.0322913)
  393546107  => ENU(-258.402, 113.932, -0.00624562)
  32547299   => ENU(-220.929, 125.097, -0.00504936)
  393551222  => ENU(-13.0844, 661.744, -0.0344082)
  6294585469 => ENU(512.134, -209.903, -0.0239888)
  1895652027 => ENU(-508.215, 721.56, -0.061109)
  6294585463 => ENU(456.838, -228.439, -0.0204333)
  6509238509 => ENU(-220.461, 123.631, -0.00500454)
  393552046  => ENU(-380.81, 547.75, -0.0349156)
  6509238507 => ENU(-221.429, 126.597, -0.00509632)
  469107079  => ENU(-155.844, -409.655, -0.0150819)
  7137874035 => ENU(-243.219, -88.4244, -0.00524407)
  6509260328 => ENU(-267.934, 19.0377, -0.0056472)
  29687649   => ENU(-233.151, -122.523, -0.00

In [25]:
ttc_data = """
lat	lon	name
43.656136534	-79.380729654	Dundas
43.652346514	-79.379326503	Queen
43.649120879	-79.378045133	King
43.645722666	-79.380462258	Union
43.647645649	-79.385130483	St Andrew
43.650874106	-79.386617316	Osgoode
43.654611638	-79.388295977	St Patrick
""";# Sample file with real locations

"lat\tlon\tname\n43.656136534\t-79.380729654\tDundas\n43.652346514\t-79.379326503\tQueen\n43.649120879\t-79.378045133\tKing\n43.645722666\t-79.380462258\tUnion\n43.647645649\t-79.385130483\tSt Andrew\n43.650874106\t-79.386617316\tOsgoode\n43.654611638\t-79.388295977\tSt Patrick\n"

In [26]:
ttc = CSV.File(IOBuffer(ttc_data)) |> DataFrame

Row,lat,lon,name
,Float64,Float64,String15
1,43.6561,-79.3807,Dundas
2,43.6523,-79.3793,Queen
3,43.6491,-79.378,King
4,43.6457,-79.3805,Union
5,43.6476,-79.3851,St Andrew
6,43.6509,-79.3866,Osgoode
7,43.6546,-79.3883,St Patrick


In [32]:
function add_graph_edge!(m::MapData, va::Int, vb::Int;symmetric=false)
    LightGraphs.add_edge!(m.g, va, vb)
    push!(m.e,(m.n[va], m.n[vb]) )
    symmetric && add_graph_edge!(m, vb, va; symmetric=false)
end

wI, wJ, wV = findnz(m.w) 
agent_speed = 3_000/60 # meters in minute (that  is 3km/h)
wV = wV ./ agent_speed # all distances in minutes instead of meters
points = [ENU(LLA(ttc.lat[t], ttc.lon[t]), m.bounds) for t in 1:nrow(ttc)]
nodecs = nearest_node.(Ref(m), points)

for t in 1:nrow(ttc)
    LightGraphs.add_vertex!(m.g)
    vx = nv(m.g)        
    push!(m.n, -vx) # add vertex to MapData - negative indices are used for special purposes
                   #for simplicity we assume no identifier clash
    m.nodes[-vx] = points[t]
    m.v[-vx] = vx
    vc = m.v[nodecs[t]]
    
    add_graph_edge!(m, vc, vx; symmetric=true)
    
    append!(wI, [vx, vc])
    append!(wJ, [vc, vx])
    append!(wV, [3.0, 3.0]) # getting in or out of TTC takes 3 minutes    
    if t>1  #build the TTC line in graph 
        add_graph_edge!(m, vx-1, vx; symmetric=false)
        append!(wI, vx-1)
        append!(wJ, vx)
        append!(wV, 2.0) # TTC travels 2 minutes between stops 
                                # this is a simplificatio of course real data
                                # could be used
        
    end
    
end

# now we construct the updated distance matrix:
m.w = sparse(wI,wJ,wV)
@assert size(m.w,1)==size(m.w,2)
@assert size(m.w,1)==nv(m.g)

MethodError: MethodError: no method matching add_vertex!(::Graphs.SimpleGraphs.SimpleDiGraph{Int64})
Closest candidates are:
  add_vertex!(!Matched::SimpleDiGraph{T}) where T at ~/.julia/packages/LightGraphs/IgJif/src/SimpleGraphs/simpledigraph.jl:449
  add_vertex!(!Matched::SimpleGraph{T}) where T at ~/.julia/packages/LightGraphs/IgJif/src/SimpleGraphs/simplegraph.jl:526

In [33]:
using LightGraphs